## Neural Networks (2022-2023)
### Lab session 1: Logistic regression in PyTorch

In [1]:
import torch

#### Part 1: Tensor computations

In [2]:
# PyTorch syntax is very similar to NumPy
X = torch.arange(6)
X

tensor([0, 1, 2, 3, 4, 5])

In [3]:
# A tensor is described by its type and shape.
print(X.dtype)
print(X.shape)

torch.int64
torch.Size([6])


In [4]:
# In addition, any tensor lives on a certain device, and operations
# are only possible for tensors staying on the same device (not an
# issue for now).
X.device

device(type='cpu')

In [5]:
# No GPUs for the moment!
torch.cuda.device_count()

0

In [6]:
# In caso of a GPU, this is needed to send the tensor on the GPU.
# Otherwise, this can be specified in the constructor itself.
# X.to('cuda')

In [7]:
# Reshaping converts the tensor to a new tensor having a different shape.
# In some cases, it is possible to use a more efficient view operation, provided
# the view is compatible with the underlying data storage:
# https://pytorch.org/docs/stable/generated/torch.Tensor.view.html#torch.Tensor.view
y = X.reshape((2, 3))
y.shape

torch.Size([2, 3])

In [8]:
# Broadcasting can also be used in PyTorch:
# https://pytorch.org/docs/stable/notes/broadcasting.html
(X + torch.rand((3, 6))).shape

torch.Size([3, 6])

In [9]:
# Double broadcasting!
(X + X.reshape(6,1)).shape

torch.Size([6, 6])

In [10]:
#Indexing is also similar to numpy, let's build a tensor and try indexing it
X = torch.arange(20).reshape(5,4)
X

tensor([[ 0,  1,  2,  3],
        [ 4,  5,  6,  7],
        [ 8,  9, 10, 11],
        [12, 13, 14, 15],
        [16, 17, 18, 19]])

In [11]:
#You can index tensors in many different ways
#multidimensional indexing
X[0, 1]

tensor(1)

In [12]:
#indexing a whole column
X[:,-1]

tensor([ 3,  7, 11, 15, 19])

In [13]:
#the first two rows
X[:2]

tensor([[0, 1, 2, 3],
        [4, 5, 6, 7]])

In [14]:
#only even rows
X[::2]

tensor([[ 0,  1,  2,  3],
        [ 8,  9, 10, 11],
        [16, 17, 18, 19]])

In [15]:
#Integer indexing: you can index with data from another tensor or list

X[[0,2,4,0], [1,3,0,1]]

tensor([ 1, 11, 16,  1])

In [16]:
#Masking: you can use boolean masks to index your tensors, this allows for pretty compact expressions
#let's take only values of X divisible by 3
X[X%3==0]

tensor([ 0,  3,  6,  9, 12, 15, 18])

In [ ]:
#how did that work in detail? Look at X%3==0
X%3 == 0

tensor([[ True, False, False,  True],
        [False, False,  True, False],
        [False,  True, False, False],
        [ True, False, False,  True],
        [False, False,  True, False]])

In [ ]:
# Simple matrix multiplication
X @ X.T

tensor([[  14,   38,   62,   86,  110],
        [  38,  126,  214,  302,  390],
        [  62,  214,  366,  518,  670],
        [  86,  302,  518,  734,  950],
        [ 110,  390,  670,  950, 1230]])

In [ ]:
# Reductions operate on the entire tensor by default,
X.sum()

tensor(190)

In [ ]:
# unless a dimension is specified.
X.sum(dim=0)

tensor([40, 45, 50, 55])

#### Part 2: Automatic differentiation

In [17]:
# the main difference from numpy is the possibility of setting requires_grad=True, meaning that all operations on X will be tracked in order to compute gradients.
X = torch.randn((3, 2), requires_grad=True)
print(X)

tensor([[-0.8927,  0.2274],
        [-0.5980, -0.3699],
        [-1.6493, -0.4060]], requires_grad=True)


In [18]:
# Anything is okay here, provided the output is a scalar.
y = ((X.cos()) @ X.T**2).sum()

In [19]:
# The grad_fn property allows to reconstruct the computational graph.
y

tensor(6.3274, grad_fn=<SumBackward0>)

In [20]:
# This will compute all gradients of y w.r.t. tensors involved in its computation
# that have requires_grad = True.
y.backward()

In [21]:
# Gradients are accumulated inside the grad property of the tensors.
X.grad

tensor([[ 0.5619,  1.2053],
        [ 0.5365, -1.9625],
        [-0.6740, -2.1544]])

In [ ]:
# Equivalent to the above instruction, but gradients are returned.
# torch.autograd.grad(y, [X])

In [ ]:
# The computational graph is freed after the first backward call.
# Running it again is impossible, try it.
# y.backward()

In [ ]:
# If you do not want some operation to be traced, you need to include
# it in a proper no_grad context.
with torch.no_grad():
  y = X.sum()

In [ ]:
print(y.grad_fn)

None


#### Exercises 💪

1)Look at the transpose (https://pytorch.org/docs/stable/generated/torch.transpose.html) and reshape (https://pytorch.org/docs/stable/generated/torch.reshape.html) operations and consider (and run!) the following lines:

```
a = torch.arange(12).reshape(3,4)
b = a.transpose(0,1)
c = a.reshape(4,3)
```
Are b and c the same tensor? If not, what's the difference?

2)Create a 3x3 matrix that is filled with 2 on the diagonal and 1 elsewhere.
Hint: https://pytorch.org/docs/stable/generated/torch.eye.html

3)Create a 4x4 matrix whose element i,j is defined as
\begin{cases}
 i+j, & \text{if } i+j \text{ is even}\\
 -1,              & \text{otherwise}
\end{cases}

4)Consider the following expression:
\begin{equation}
k = 3x^2 - yz
\end{equation}
compute $∇k = (\frac{\partial k}{\partial x},\frac{\partial k}{\partial y},\frac{\partial k}{\partial z})$ and use automatic differentiation from pytorch to verify it for some values of x,y,z.

In [ ]:
#Exercise 1
a = torch.arange(12).reshape(3,4)
b = a.transpose(0,1)
c = a.reshape(4,3)
print(a)
print(b)
print(c)

print("The tensors are different! transpose swaps columns and rows so that a(i,j) becomes b(j,i), reshape instead flattens the array and then rebuilds an array of the desired shape")

tensor([[ 0,  1,  2,  3],
        [ 4,  5,  6,  7],
        [ 8,  9, 10, 11]])
tensor([[ 0,  4,  8],
        [ 1,  5,  9],
        [ 2,  6, 10],
        [ 3,  7, 11]])
tensor([[ 0,  1,  2],
        [ 3,  4,  5],
        [ 6,  7,  8],
        [ 9, 10, 11]])
The tensors are different! transpose swaps columns and rows so that a(i,j) becomes b(j,i), reshape instead flattens the array and then rebuilds an array of the desired shape


In [ ]:
#Exercise 2
#We can use torch.eye + broadcasting a scalar to the whole matrix to obtain the desired matrix
torch.eye(3)+1

tensor([[2., 1., 1.],
        [1., 2., 1.],
        [1., 1., 2.]])

In [ ]:
#Exercise 3
#We use double broadcasting to build a matrix where each element contains (i+j)
matr = torch.arange(4) + torch.arange(4).reshape(4,1)
#Then we use boolean masking to set to -1 the right cells
matr[matr%2!=0] = -1
matr

tensor([[ 0, -1,  2, -1],
        [-1,  2, -1,  4],
        [ 2, -1,  4, -1],
        [-1,  4, -1,  6]])

In [ ]:
#Exercise 4
#We expect the gradient to be (6x, -z, -y), let's verify it
x = torch.tensor(2., requires_grad=True)
y = torch.tensor(3., requires_grad=True)
z = torch.tensor(4., requires_grad=True)

k = 3*x*x - y*z
k.backward()
print(x.grad, y.grad, z.grad)

tensor(12.) tensor(-4.) tensor(-3.)


### Part 3: Training a simple Logistic regression in Pytorch



#### Part 3.1 Data 📊

We are going to use the [Palmer Penguins dataset](https://allisonhorst.github.io/palmerpenguins/articles/intro.html).

Given the features of a penguin (bill depth, body mass, ...) ***the goal is to predict the species*** among three possible options: *Chinstrap, Gentoo, Adélie*.


<div>
<img src="https://allisonhorst.github.io/palmerpenguins/reference/figures/lter_penguins.png" width="500"/>
</div>

In [ ]:
import pandas as pd

# We use a preprocessed version of the dataset
# The same version can be found at https://www.tensorflow.org/datasets/catalog/penguins
# Out goal is to predict the species based on the features!

penguins = pd.read_csv('https://raw.githubusercontent.com/alessiodevoto/notebooks/main/data/normalized_palmer_penguins.csv', index_col=False)

In [ ]:
penguins

,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,species,species_str
0,0.654545,0.226190,0.898305,0.638889,2,Adelie
1,0.360000,0.047619,0.644068,0.402778,2,Adelie
2,0.680000,0.309524,0.915254,0.694444,2,Adelie
3,0.618182,0.202381,0.813559,0.680556,2,Adelie
4,0.552727,0.261905,0.847458,0.708333,2,Adelie
...,...,...,...,...,...,...
329,0.327273,0.607143,0.338983,0.375000,0,Chinstrap
330,0.669091,0.250000,0.745763,0.638889,2,Adelie
331,0.258182,0.654762,0.305085,0.430556,0,Chinstrap
332,0.152727,0.761905,0.305085,0.305556,0,Chinstrap


In [ ]:
import plotly.express as px

# Some visualizations ?
# A simple bar plot

px.bar(penguins, x='species_str', color='species_str', width=600, height=500)



In [ ]:
# or a scatter plot
px.scatter(penguins, x='bill_length_mm', y='bill_depth_mm', color='species_str', width=600, height=500)

In [ ]:
# we don't need the string column for the species
penguins = penguins.drop('species_str', axis=1)

In [ ]:
penguins

,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,species
0,0.654545,0.226190,0.898305,0.638889,2
1,0.360000,0.047619,0.644068,0.402778,2
2,0.680000,0.309524,0.915254,0.694444,2
3,0.618182,0.202381,0.813559,0.680556,2
4,0.552727,0.261905,0.847458,0.708333,2
...,...,...,...,...,...
329,0.327273,0.607143,0.338983,0.375000,0
330,0.669091,0.250000,0.745763,0.638889,2
331,0.258182,0.654762,0.305085,0.430556,0
332,0.152727,0.761905,0.305085,0.305556,0


In [ ]:
# now we can create a train and a test data
# we only keep the features for the X and the species for the Y
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(penguins.drop('species', axis=1), penguins['species'], stratify=penguins['species'], test_size=0.2)

In [ ]:
x_train

,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g
183,0.549091,0.071429,0.711864,0.618056
258,0.236364,0.488095,0.457627,0.291667
190,0.225455,0.726190,0.288136,0.347222
189,0.327273,0.702381,0.169492,0.201389
158,0.141818,0.559524,0.389831,0.208333
...,...,...,...,...
288,0.614545,0.761905,0.644068,0.347222
277,0.090909,0.952381,0.440678,0.472222
30,0.556364,0.178571,0.677966,0.562500
156,0.563636,0.619048,0.389831,0.319444


In [ ]:
# Let's go back to PyTorch!
# We should create a Pytorch tensor holding the same values as the pandas Dataframe
import torch

X_train = torch.tensor(x_train.values).float()

In [ ]:
X_train

tensor([[0.5491, 0.0714, 0.7119, 0.6181],
        [0.2364, 0.4881, 0.4576, 0.2917],
        [0.2255, 0.7262, 0.2881, 0.3472],
        ...,
        [0.5564, 0.1786, 0.6780, 0.5625],
        [0.5636, 0.6190, 0.3898, 0.3194],
        [0.2764, 0.5476, 0.3559, 0.1389]])

In [ ]:
# instead of a pandas dataframe we now have a big tensor holding the same values
# check the shape
X_train.shape

torch.Size([267, 4])

In [ ]:
# now the first penguin 🐧 is the first row of our tensor
X_train[0]

tensor([0.5491, 0.0714, 0.7119, 0.6181])

In [ ]:
# We should represent all as pytorch tensors
X_train, X_test, y_train, y_test = torch.tensor(x_train.values).float(), torch.tensor(x_test.values).float(), torch.tensor(y_train.values), torch.tensor(y_test.values)

#### Part 3.2 Logistic regression 📈

A logistic regression maps the input features of our samples into a smaller number of output features. The smaller representation (made of logistic units) is then used for a classification through the logistic function (sigmoid, softmax ...)

For an input sample with $x_n$ features and a single output it looks like this.

![image](https://www.researchgate.net/profile/Mohammad-Khan-48/publication/352975798/figure/fig4/AS:1080307235205134@1634576724565/Flowchart-of-logistic-regression.jpg)



We want to do the same but:
- we have 3 outputs (i.e. the number of output classes)
- we have 233 penguins to classify !

$$ \Large σ(x W + b)$$

where

- $ \sigma$ is the softmax function
- $ W, b$ are weights and biases
- $ x $ are the input features (out penguins 🐧)

In [ ]:
# This initializes all parameters for the logistic regression model.
# Big question: what size should the W and b tensor be ?
# Note the requires_grad = True on both.

def init():
  W = torch.randn((4, 3), requires_grad=True)
  b = torch.randn((3,), requires_grad=True)
  return W, b

In [ ]:
# Finally let us code the logistic regression function!
# It will be just a line

def logreg(X, W, b):
  """ A logistic regression model.
  Inputs:
  - X (n, 4): input matrix for the model.
  - W (4, 3): weight matrix.
  - b (3,): bias vector.

  Returns a (n, 3) output.
  """
  return torch.softmax(X @ W + b, dim=1)

In [ ]:
# Just to double check that everything is working, let us run a simple prediction step

# initialize the weights
W, b = init()

# get a (trash) result
y_pred = logreg(X_train, W, b)




In [ ]:
# visualize predictions for the first penguin
# how should we interpret it ?
y_pred[0]

tensor([0.0012, 0.4476, 0.5512], grad_fn=<SelectBackward0>)

#### Part 3.2 Loss and accuracy

How can we tell the model if it doing right or wrong predictions ? We need a function for loss and accuracy to figure it out!

We are going to use the **Cross Entropy loss**.

$$ \Large H(y, p) = - \sum_{i=1}^N y_i \cdot \log(p_i)$$

where

- $y_i$ is the label for class $i$, i.e. $1$ or $0$
- $p_i$ is the model's prediction for class $i$ in $[0, 1]$

In [ ]:
# given ypred
print(y_pred[:5, :])

# and ytrue
print(y_train[:5])


tensor([[0.0012, 0.4476, 0.5512],
        [0.0020, 0.4697, 0.5283],
        [0.0018, 0.4969, 0.5013],
        [0.0020, 0.4603, 0.5377],
        [0.0024, 0.4656, 0.5321]], grad_fn=<SliceBackward0>)
tensor([2, 0, 0, 0, 0])


In [ ]:
# how can I compute the formula ?
# we want to compute the logarithm only for the correct class
y_pred[torch.arange(0, y_pred.shape[0]), y_train][:5]


tensor([0.5512, 0.0020, 0.0018, 0.0020, 0.0024], grad_fn=<SliceBackward0>)

In [ ]:

def cross_entropy(ytrue, y_pred):
  """ Cross-entropy loss.
  Inputs:
  - ytrue (n,): vector of indices for the correct class.
  - ypred (n, 3): predictions of the model.
  Returns the average cross-entropy.
  """
  # This is called integer array indexing in NumPy:
  # https://numpy.org/doc/stable/user/basics.indexing.html#integer-array-indexing
  return - y_pred[torch.arange(0, y_pred.shape[0]), ytrue].log().mean()

In [ ]:
ypred = logreg(X_train, W, b)
cross_entropy(y_train, ypred)

tensor(3.1334, grad_fn=<NegBackward0>)

What about the **accuracy** ?

$$ \Large Accuracy(y, p) = \frac{\#correct \ predictions}{\#total \ predictions}$$

In [ ]:
# we want to check in how many cases ypred most likely prediction equals the ground truth

# this is ypred
print(y_pred[:5, :].argmax(1))

# we can use the arrgmax on the first dimension !
print(y_train[:5])



tensor([2, 2, 2, 2, 2])
tensor([2, 0, 0, 0, 0])


In [ ]:
def accuracy(ytrue, y_pred):
  return (y_pred.argmax(1) == ytrue).float().mean()

In [ ]:
accuracy(y_train, ypred)

tensor(0.2921)

#### Part 3.4 Finally train the logistic regression 🔥

In [ ]:
W, b = init()

In [ ]:
losses = []
accuracies = []

for i in range(5000):

  # Compute predictions
  y_pred = logreg(X_train, W, b)

  # Compute the loss
  loss = cross_entropy(y_train, y_pred)

  # Compute the gradients
  loss.backward()

  # where are the gradients?

  with torch.no_grad():

    # Update weights by applying gradients
    W -= 0.01 * W.grad
    b -= 0.01 * b.grad

    # Gradients are accumulated: we need to zero them out before the next iteration.
    W.grad.zero_()
    b.grad.zero_()

    # Keep track of losses and accuracies to plot them later
    losses.append(loss.detach().item())

    acc = accuracy(y_train, y_pred).item()
    accuracies.append(acc)

In [ ]:
px.line(losses, width=600, height=500)

In [ ]:
px.line(losses, width=600, height=500)

In [ ]:
px.line(accuracies, width=600, height=500)

In [ ]:
px.line(accuracies, width=600, height=500)

#### Part 4 💪 Exercises

1. We have not used the test portion of the dataset, modify the training loop to include tracking of test loss and test accuracy.

2. Add more metrics to be tracked, e.g., multi-class F1-score. The torchmetrics package has a lot of options in this sense: https://torchmetrics.readthedocs.io/en/stable/.

3. Momentum is a simple technique to improve the convergence speed of gradient descent. The key idea is to update each variable using a weighted average of the current gradient, and the gradient at the previous iteration (see Section 12.6 in the book). The weighting parameter is called the momentum weight. Implement momentum in the codelab, using a weight of 0.5.

##### Test set

In [ ]:
W, b = init()

losses, test_losses = [], []
accuracies, test_accuracies = [], []

for i in range(5000):

  # Compute predictions
  y_pred = logreg(X_train, W, b)

  # Compute the loss
  loss = cross_entropy(y_train, y_pred)

  # Compute the gradients
  loss.backward()

  # where are the gradients?

  with torch.no_grad():

    # Update weights by applying gradients
    W -= 0.01 * W.grad
    b -= 0.01 * b.grad

    # Gradients are accumulated: we need to zero them out before the next iteration.
    W.grad.zero_()
    b.grad.zero_()

    # Keep track of losses and accuracies to plot them later
    losses.append(loss.detach().item())

    acc = accuracy(y_train, y_pred).item()
    accuracies.append(acc)

  # here we add the validation on the test set
  with torch.no_grad():
    y_pred_t = logreg(X_test, W, b)
    loss = cross_entropy(y_test, y_pred_t)
    acc = accuracy(y_test, y_pred_t).item()
    test_losses.append(loss)
    test_accuracies.append(acc)


In [ ]:
px.line(test_losses, width=600, height=500)

In [ ]:
px.line(test_accuracies, width=600, height=500)

##### Metrics

In [ ]:
!pip install torchmetrics

In [ ]:
import torchmetrics
f1 = torchmetrics.classification.F1Score(task="multiclass", num_classes=3)

W, b = init()

losses, test_losses = [], []
accuracies, test_accuracies = [], []
f1s = []

for i in range(5000):

  # Compute predictions
  y_pred = logreg(X_train, W, b)

  # Compute the loss
  loss = cross_entropy(y_train, y_pred)

  # Compute the gradients
  loss.backward()

  # where are the gradients?

  with torch.no_grad():

    # Update weights by applying gradients
    W -= 0.01 * W.grad
    b -= 0.01 * b.grad

    # Gradients are accumulated: we need to zero them out before the next iteration.
    W.grad.zero_()
    b.grad.zero_()

    # Keep track of losses and accuracies to plot them later
    losses.append(loss.detach().item())

    acc = accuracy(y_train, y_pred).item()
    accuracies.append(acc)


  with torch.no_grad():
    y_pred_t = logreg(X_test, W, b)
    loss = cross_entropy(y_test, y_pred_t)
    acc = accuracy(y_test, y_pred_t).item()
    f1_score = f1(y_pred_t, y_test)

    f1s.append(f1_score)
    test_losses.append(loss)
    test_accuracies.append(acc)





In [ ]:
px.line(f1s, width=600, height=500)

##### Momentum

In [ ]:
losses = []
accuracies = []

update_W, update_b = 0, 0
decay = 0.5

for i in range(5000):

  # Compute predictions
  y_pred = logreg(X_train, W, b)

  # Compute the loss
  loss = cross_entropy(y_train, y_pred)

  # Compute the gradients
  loss.backward()

  # where are the gradients?

  with torch.no_grad():

    # Update weights by applying gradients

    update_W = decay * update_W + 0.01 * W.grad
    update_b = decay * update_b + 0.01 * b.grad
    W -= update_W
    b -= update_b

    # Gradients are accumulated: we need to zero them out before the next iteration.
    W.grad.zero_()
    b.grad.zero_()

    # Keep track of losses and accuracies to plot them later
    losses.append(loss.detach().item())

    acc = accuracy(y_train, y_pred).item()
    accuracies.append(acc)